# 生成器 generator
一种特殊的迭代器 iterator，在需要时动态生成值，实现“惰性计算”

## 列表推导式与生成器表达式
- 大型数据集或流数据时，生成器表达式更优，节省内存
  - 如处理文件字节长度 sum(x for x in (len(line) for line in open(filename)))
- 小型数据集，列表推导式更快，可一次性生成所有值

In [ ]:
## 1. 列表推导式 <class 'list'>
lst_generator = [0 for _ in range(10)] # 10 个元素为0 的列表
print(f'lst_generator: {lst_generator}, 其类型为{type(lst_generator)}')

## 2. 生成器表达式 <class 'generator'>，与列表推导式类似，但返回的是生成器对象而不是列表
generator_expression = (0 for _ in range(10))
print(f'generator_expression: {generator_expression}, 其类型为{type(generator_expression)}')

print(next(generator_expression))
print(next(generator_expression))
# 注意 next 次数不可超过 generator_expression 的长度，否则抛错 StopIteration

## 3. 生成器函数的使用
# 使用 yield 返回值，不是 return
# 使用生成器函数会返回一个迭代器，可以使用 next() 函数获取值

def my_generator():
    for i in range(10):
        yield i

gen = my_generator()
print(next(gen)) # 0
print(next(gen)) # 1

# 生成器双向通信：不仅可以向调用者返回值，还可以从调用者接收值（PS：进阶：还支持 throw 和 close 方法）
def send_generator_for_plus():
    x = yield
    while True:
        x = yield x + x

sg = send_generator_for_plus()
next(sg)
print(sg.send(10))
print(sg.send(3))





lst_generator: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 其类型为<class 'list'>
generator_expression: <generator object <genexpr> at 0x10f463510>, 其类型为<class 'generator'>
0
0
0
1
20
6


# Python 管理内存
https://docs.python.org/zh-tw/3.11/library/tracemalloc.html
- objgraph
- memory_profiler
- tracemalloc

## tracemalloc
tracemalloc 模块是一个用于对 python 已申请的内存快进行 debug 的工具。它能以供一下信息：
- 回溯对象分配内存的位置
- 按文件、行统计 python 的内存块分配情况：内存块总大小、数量以及块平均大小
- 对比两个内存快照的差异，以便排查内存泄露

### 显示前 10 项

In [18]:
import tracemalloc

tracemalloc.start()
a = [i for i in range(10000)]
print('hello world')
snapshot = tracemalloc.take_snapshot()
top_stats = snapshot.statistics('lineno')

print('[Top 10 malloc]')
for stat in top_stats[:10]:
    print(stat)

hello world
[Top 10 malloc]
/var/folders/wy/sv7xp55526xfh1ky9r2fcyy40000gn/T/ipykernel_27539/1615604996.py:4: size=388 KiB, count=9745, average=41 B
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/IPython/core/compilerop.py:174: size=38.7 KiB, count=342, average=116 B
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/tracemalloc.py:558: size=22.1 KiB, count=405, average=56 B
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/tracemalloc.py:115: size=17.7 KiB, count=226, average=80 B
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/tracemalloc.py:193: size=15.1 KiB, count=323, average=48 B
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/tracemalloc.py:125: size=13.9 KiB, count=198, average=72 B
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/IPython/core/compilerop.py:86: size=13.8 KiB, count=158, average=89 B
/Library/Frameworks/Python.framework/Versions/3.1

### 计算差异
获取两个快照并显示差异

In [42]:
import tracemalloc

tracemalloc.start()
a = [i for i in range(5)]
print('hello world')
snapshot1 = tracemalloc.take_snapshot()
b = (i for i in range(2**32))
snapshot2 = tracemalloc.take_snapshot()

top_stats = snapshot2.compare_to(snapshot1, 'lineno')

print('[Top 10 malloc]')
for stat in top_stats[:10]:
    print(stat)

hello world
[Top 10 malloc]
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/tracemalloc.py:558: size=73.2 KiB (+27.9 KiB), count=1416 (+523), average=53 B
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/tokenize.py:532: size=0 B (-8568 B), count=0 (-153)
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/pygments/regexopt.py:77: size=952 B (-8064 B), count=17 (-144), average=56 B
<frozen genericpath>:80: size=0 B (-6104 B), count=0 (-109)
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/tracemalloc.py:193: size=2208 B (-4128 B), count=46 (-86), average=48 B
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/codeop.py:118: size=3398 B (+502 B), count=47 (+6), average=72 B
/var/folders/wy/sv7xp55526xfh1ky9r2fcyy40000gn/T/ipykernel_27539/796183124.py:7: size=400 B (+400 B), count=3 (+3), average=133 B
<frozen genericpath>:77: size=41.1 KiB (-352 B), count=388 (-6), average=109 B
/Library/Frame

### 获取一个内存块的溯源 code
一段找出程序中最大内存块溯源的代码